# TensorFlow

In [0]:
!pip install -q --upgrade tensorflow

     |████████████████████████████████| 86.3MB 131kB/s 
     |████████████████████████████████| 450kB 52.3MB/s 
     |████████████████████████████████| 3.8MB 57.8MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.


In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.0.0'

In [0]:
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import time
import cv2
from skimage.measure import label
import matplotlib.pyplot as plt

# Dataset

In [0]:
import urllib
import pandas as pd
import tarfile
import os
import matplotlib.pyplot as plt

In [0]:
links = [
  'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
  'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
  'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
  'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
  'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

In [0]:
for idx, link in enumerate(links):
    fn = 'images_%02d.tar.gz' % (idx+1)
    print('downloading', fn, '...')
    urllib.request.urlretrieve(link, fn)  # download the zip file

print("Download complete. Please check the checksums")

downloading images_01.tar.gz ...
downloading images_02.tar.gz ...
downloading images_03.tar.gz ...
downloading images_04.tar.gz ...
downloading images_05.tar.gz ...
downloading images_06.tar.gz ...
downloading images_07.tar.gz ...
downloading images_08.tar.gz ...
downloading images_09.tar.gz ...
downloading images_10.tar.gz ...
downloading images_11.tar.gz ...
downloading images_12.tar.gz ...
Download complete. Please check the checksums


In [0]:
for index in range(1, 10):
  file_name = f'images_0{index}.tar.gz'
  tarf = tarfile.open(file_name)
  tarf.extractall()
  print(f"{file_name} completed")

images_01.tar.gz completed
images_02.tar.gz completed
images_03.tar.gz completed
images_04.tar.gz completed
images_05.tar.gz completed
images_06.tar.gz completed
images_07.tar.gz completed
images_08.tar.gz completed
images_09.tar.gz completed


In [0]:
for index in range(10, 13):
  file_name = f'images_{index}.tar.gz'
  tarf = tarfile.open(file_name)
  tarf.extractall()
  print(f"{file_name} completed")

images_10.tar.gz completed
images_11.tar.gz completed
images_12.tar.gz completed


## Load original csv

In [0]:
data_csv = pd.read_csv("Data_Entry_2017.csv")

In [0]:
data_csv.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [0]:
data_csv["Patient ID"].unique().size

30805

In [0]:
training_size = 112120 * .70
validation_size = 112120 * .10
test_size = 112120 * .20

In [0]:
training_size, validation_size, test_size

(78484.0, 11212.0, 22424.0)

In [0]:
training_split = data_csv.sample(n=int(training_size))

In [0]:
data_csv["Image Index"].count()

112120

In [0]:
training_split = data_csv.sample(n=int(training_size), replace=False)
validation_split = data_csv.drop(training_split.index).sample(n=int(validation_size), replace=False)
test_split = data_csv.drop(training_split.index).drop(validation_split.index)

In [0]:
training_split["Image Index"].count(), validation_split["Image Index"].count(), test_split["Image Index"].count()

(78484, 11212, 22424)

In [0]:
training_split.to_csv('training_split.csv')
validation_split.to_csv('validation_split.csv')
test_split.to_csv('test_split.csv')

## Load split csv

In [0]:
training_split = pd.read_csv("training_split.csv")
validation_split = pd.read_csv("validation_split.csv")
test_split = pd.read_csv("test_split.csv")

In [0]:
classes_names = {
  "Atelectasis": 0,
  "Consolidation": 1,
  "Infiltration": 2,
  "Pneumothorax": 3,
  "Edema": 4,
  "Emphysema": 5,
  "Fibrosis": 6,
  "Effusion": 7,
  "Pneumonia": 8,
  "Pleural_Thickening": 9,
  "Cardiomegaly": 10,
  "Nodule": 11,
  "Mass": 12,
  "Hernia": 13,
  "No Finding": 14,
}

In [0]:
def load_labels(text_labels):
  labels = np.zeros((15), dtype=int)

  text_labels = text_labels.split("|")

  for true_label in text_labels:
    class_index = classes_names[true_label]
    labels[class_index] = 1

  return labels

### Load training data

In [0]:
train_labels = second_training_split["Finding Labels"].values
train_images_path = second_training_split["Image Index"].values

In [0]:
train_labels.size, train_images_path.size

(19620, 19620)

In [0]:
onehot_train_labels = list(map(load_labels, train_labels))

In [0]:
onehot_train_labels = np.array(onehot_train_labels)

In [0]:
train_images_path = np.array([f"images/{img_path}" for img_path in train_images_path])

### Load validation data

In [0]:
val_labels = validation_split["Finding Labels"].values
val_images_path = validation_split["Image Index"].values

In [0]:
val_labels.size, val_images_path.size

(11212, 11212)

In [0]:
onehot_val_labels = list(map(load_labels, val_labels))

In [0]:
onehot_val_labels = np.array(onehot_val_labels)

In [0]:
val_images_path = np.array([f"images/{img_path}" for img_path in val_images_path])

## Generator

In [0]:
image_size = (224, 224, 3)

In [0]:
@tf.function()
def random_jitter(img):
  resized_image = tf.image.resize(img, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  
  cropped_image = tf.image.random_crop(resized_image, size=[image_size[0], image_size[1], 3])

  if tf.random.uniform(()) > 0.5:
    cropped_image = tf.image.flip_left_right(cropped_image)

  return cropped_image

In [0]:
@tf.function()
def crop_center(img):
  resized_image = tf.image.resize(img, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  cropped_image = tf.image.central_crop(resized_image, 0.125)

  return cropped_image

In [0]:
def normalize(img):
  imagenet_mean = np.array([103.939, 116.779, 123.68])
  img = tf.subtract(img, imagenet_mean)

  return img

In [0]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.cast(img, tf.float32)

  return img

In [0]:
def load_train_data(image_path, labels):
  img = load_image(image_path)
  img = random_jitter(img)
  img = normalize(img)

  img = tf.cast(img, tf.float32)

  return img, labels

In [0]:
def load_val_data(image_path, labels):
  img = load_image(image_path)
  img = crop_center(img)
  img = normalize(img)

  return img, labels

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, onehot_train_labels))
train_dataset = train_dataset.shuffle(len(onehot_train_labels))
train_dataset = train_dataset.map(load_train_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_images_path, onehot_val_labels))
val_dataset = val_dataset.shuffle(len(onehot_val_labels))
val_dataset = val_dataset.map(load_val_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Models

In [0]:
num_classes = 15

In [0]:
def create_branch():
  dense_model = DenseNet121(
      weights="imagenet",
      input_shape=image_size,
      include_top=False)

  feature_maps = dense_model.output
  pooling_layer = layers.GlobalMaxPool2D()(feature_maps)
  predictions = layers.Dense(num_classes, activation="sigmoid")(pooling_layer)

  model = Model(inputs=dense_model.input, outputs=[predictions, feature_maps, pooling_layer])

  weight_decay_value = 1e-4
  weight_decay = tf.keras.regularizers.l2(l=weight_decay_value)

  for layer in model.layers:
    layer.trainable = True

    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = weight_decay

  return model

In [0]:
global_branch_model = create_branch()

29089792/29084464 [==============================] - 1s 0us/step


In [0]:
global_branch_model.summary()

In [0]:
local_branch_model = create_branch()

In [0]:
def create_fusion_branch():
  global_pool = layers.Input(shape=(1024))
  local_pool = layers.Input(shape=(1024))

  concat_layer = layers.concatenate([global_pool, local_pool])
  model = layers.Dense(num_classes, activation="sigmoid")(concat_layer)

  model = Model(inputs=[global_pool, local_pool], outputs=model)

  return model

In [0]:
fusion_branch_model = create_fusion_branch()

In [0]:
fusion_branch_model.summary()

# Training

In [0]:
def compute_loss(true_classes, pred_classes):
  loss = tf.keras.losses.binary_crossentropy(true_classes, pred_classes)
  return tf.reduce_mean(loss)

In [0]:
auc_metric = tf.keras.metrics.AUC()

In [0]:
def compute_metrics(true_classes, pred_classes):
  auc_metric.reset_states()
  auc_metric.update_state(true_classes, pred_classes)
  auc_value = auc_metric.result()

  return auc_value

In [0]:
epochs = 50

## Stage 1 (Global Model)

In [0]:
batch_size = 64
train_steps = int(train_labels.size / batch_size)
global_branch_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9)

### Checkpoints

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "stage1-ckpt")
checkpoint = tf.train.Checkpoint(global_branch_optimizer=global_branch_optimizer, global_branch_model=global_branch_model)

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/stage1-ckpt-4'

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Train functions

In [0]:
train_generator = train_dataset.batch(batch_size)

In [0]:
stage_1_loss_results = []
stage_1_auc_results = []

In [0]:
@tf.function
def train_stage_1(batch_images, labels):
    with tf.GradientTape() as tape:
      pred_classes, _, _ = global_branch_model(batch_images, training=True)
      loss = compute_loss(labels, pred_classes)
      
    auc_value = compute_metrics(labels, pred_classes)
      
    gradients = tape.gradient(loss, global_branch_model.trainable_variables)   

    global_branch_optimizer.apply_gradients(zip(gradients, global_branch_model.trainable_variables))

    return loss, auc_value

In [0]:
def train(epochs):
  for epoch in range(epochs):
    batch_time = time.time()
    epoch_time = time.time()
    step = 1

    for batch_images, labels in train_generator:
      loss, auc_value = train_stage_1(batch_images, labels)

      auc_value = auc_value.numpy()
      loss = loss.numpy()

      print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - batch_time, end='')
      
      step += 1
      stage_1_loss_results.append(loss)
      stage_1_auc_results.append(auc_value)
      
      batch_time = time.time()

    checkpoint.save(file_prefix=checkpoint_prefix)

    if (epoch + 1) % 20 == 0:
      current_lr = tf.keras.backend.get_value(global_branch_optimizer.lr)
      new_lr = current_lr / 10
      tf.keras.backend.set_value(global_branch_optimizer.lr, new_lr)

    print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - epoch_time)

In [0]:
train(epochs)

 Epoch 1/1 | Step 409/408 | loss: 0.1740795 | AUC: 0.88910556 | time: 54.43811798095703

In [0]:
checkpoint.save(file_prefix=checkpoint_prefix)

'./training_checkpoints/stage1-ckpt-4'

## Global Model Validation

In [0]:
classes_predicted = []
val_steps = int(val_labels.size / batch_size)
val_generator = val_dataset.batch(batch_size)

In [0]:
def validate():
  batch_time = time.time()
  step = 0

  for batch_images, labels in val_generator:
    pred_classes, _, _ = global_branch_model(batch_images, training=False)
    classes_predicted.extend(pred_classes.numpy())

    auc_value = compute_metrics(labels, pred_classes)

    auc_value = auc_value.numpy()

    print('\r', ' Step', f"{step}/{val_steps}", "| AUC:", auc_value, "| time:", time.time() - batch_time, end='')
    
    step += 1
    batch_time = time.time()

In [0]:
validate()

 | Step 175/175 | AUC: 0.5967078 | time: 0.2941281795501709

In [0]:
classes_predicted = np.array(classes_predicted)
classes_predicted.shape

(11212, 15)

In [0]:
auc = compute_metrics(onehot_val_labels, classes_predicted)

In [0]:
auc

<tf.Tensor: id=1380020, shape=(), dtype=float32, numpy=0.7757791>

## Stage 2 (Local Model)

In [0]:
treshhold = 0.7

In [0]:
def get_local_images(feature_maps, global_images):
  batch_size = len(feature_maps)
  local_images = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)

  for index in range(batch_size):
    feature_map = feature_maps[index, ...]
    global_image = global_images[index, ...]

    l1_distance = tf.norm(feature_map, ord=1, axis=2) # outputs 7x7
    l1_distance = l1_distance - tf.keras.backend.min(l1_distance)
    heatmap = l1_distance / tf.keras.backend.max(l1_distance)

    resized_heatmap = tf.image.resize(tf.expand_dims(heatmap, 2), image_size[0:2], method=tf.image.ResizeMethod.BILINEAR)

    binary_mask = tf.where(resized_heatmap[..., 0] > treshhold, 1, 0)
    
    labeled_mask, labels_indexes = label(binary_mask, connectivity=2, background=0, return_num=True)

    elements_by_label = [np.sum(labeled_mask == label_index) for label_index in range(1, labels_indexes + 1)]

    if len(elements_by_label) == 0:
      max_connected_region = (labeled_mask == -1)
    else:
      max_label_index = np.argmax(elements_by_label) + 1
      max_connected_region = (labeled_mask == max_label_index)

    max_connected_region_stack = tf.stack([max_connected_region, max_connected_region, max_connected_region], axis=2)

    masked_image = tf.cast(max_connected_region_stack, tf.float32) * global_image
    mask_coords = np.argwhere(masked_image != 0)

    min_y = min(mask_coords[:, 0])
    min_x = min(mask_coords[:, 1])

    max_y = max(mask_coords[:, 0])
    max_x = max(mask_coords[:, 1])

    local_image = global_image[min_y:max_y, min_x:max_x, :]
    local_image = tf.image.resize(local_image, image_size[0:2], method=tf.image.ResizeMethod.BILINEAR)

    local_images[index, ...] = local_image

  return local_images

In [0]:
batch_size = 32
train_steps = int(train_labels.size / batch_size)
local_branch_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9)

In [0]:
train_generator = train_dataset.batch(batch_size)

#### Checkpoints

In [0]:
checkpoint_stage2_prefix = os.path.join(checkpoint_dir, "stage2-ckpt")
checkpoint_stage2 = tf.train.Checkpoint(local_branch_optimizer=local_branch_optimizer, local_branch_model=local_branch_model)

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Train functions

In [0]:
stage_2_loss_results = []
stage_2_auc_results = []

In [0]:
@tf.function
def train_stage_2(local_images, labels):
    with tf.GradientTape() as tape:
      pred_classes, _, _ = local_branch_model(local_images, training=True)

      loss = compute_loss(labels, pred_classes)
      
    auc_value = compute_metrics(labels, pred_classes)
      
    gradients = tape.gradient(loss, local_branch_model.trainable_variables)   

    local_branch_optimizer.apply_gradients(zip(gradients, local_branch_model.trainable_variables))

    return loss, auc_value

In [0]:
def train(epochs):
  for epoch in range(epochs):
    batch_time = time.time()
    epoch_time = time.time()
    step = 1

    for batch_images, labels in train_generator:
      _, feature_maps, _ = global_branch_model(batch_images, training=False)
      local_images = get_local_images(feature_maps, batch_images)
      
      loss, auc_value = train_stage_2(local_images, labels)

      auc_value = auc_value.numpy()
      loss = loss.numpy()

      print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - batch_time, end='')
      
      step += 1
      stage_2_loss_results.append(loss)
      stage_2_auc_results.append(auc_value)
      
      batch_time = time.time()
  
    if (epoch + 1) % 20 == 0:
      current_lr = tf.keras.backend.get_value(local_branch_optimizer.lr)
      new_lr = current_lr / 10
      tf.keras.backend.set_value(local_branch_optimizer.lr, new_lr)
  
  checkpoint_stage2.save(file_prefix=checkpoint_stage2_prefix)
  print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - epoch_time)

In [0]:
train(epochs)

 Epoch 1/1 | Step 613/613 | loss: 0.15168105 | AUC: 0.9151786 | time: 16.711588144302368

## Stage 3

In [0]:
batch_size = 32
train_steps = int(train_labels.size / batch_size)
fusion_branch_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9)

In [0]:
train_generator = train_dataset.batch(batch_size)

### Checkpoints

In [0]:
checkpoint_stage3_prefix = os.path.join(checkpoint_dir, "stage3-ckpt")
checkpoint_stage3 = tf.train.Checkpoint(fusion_branch_optimizer=fusion_branch_optimizer, fusion_branch_model=fusion_branch_model)

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Train functions

In [0]:
stage_3_loss_results = []
stage_3_auc_results = []

In [0]:
@tf.function
def train_stage_3(global_pooling_layers, local_pooling_layers, labels):
    with tf.GradientTape() as tape:
      pred_classes = fusion_branch_model([global_pooling_layers, local_pooling_layers], training=True)

      loss = compute_loss(labels, pred_classes)
      
    auc_value = compute_metrics(labels, pred_classes)
      
    gradients = tape.gradient(loss, fusion_branch_model.trainable_variables)   

    fusion_branch_optimizer.apply_gradients(zip(gradients, fusion_branch_model.trainable_variables))

    return loss, auc_value

In [0]:
def train(epochs):
  for epoch in range(epochs):
    batch_time = time.time()
    epoch_time = time.time()
    step = 1

    for batch_images, labels in train_generator:
      _, feature_maps, global_pooling_layers = global_branch_model(batch_images, training=False)
      local_images = get_local_images(feature_maps, batch_images)
       _, _, local_pooling_layers = local_branch_model(local_images, training=False)
      
      loss, auc_value = train_stage_3(global_pooling_layers, local_pooling_layers, labels)

      auc_value = auc_value.numpy()
      loss = loss.numpy()

      print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - batch_time, end='')
      
      step += 1
      stage_3_loss_results.append(loss)
      stage_3_auc_results.append(auc_value)
      
      batch_time = time.time()

    if (epoch + 1) % 20 == 0:
      current_lr = tf.keras.backend.get_value(fusion_branch_optimizer.lr)
      new_lr = current_lr / 10
      tf.keras.backend.set_value(fusion_branch_optimizer.lr, new_lr)
  
  checkpoint_stage3.save(file_prefix=checkpoint_stage3_prefix)
  print('\r', 'Epoch', f"{epoch + 1}/{epochs}", '| Step', f"{step}/{train_steps}",  '| loss:', loss, "| AUC:", auc_value, "| time:", time.time() - epoch_time)

In [0]:
train(epochs)